## Import Dependencies

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import uniform
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import check_array, check_is_fitted
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error

## Import and Explore Concrete Dataset

In [ ]:
concrete_df = pd.read_csv("concrete_data.csv")

"csMPa" is the compresive strength, the target variable.

In [ ]:
concrete_df

In [ ]:
concrete_df.describe()

In [ ]:
concrete_df.info()

In [ ]:
def plot_correlation_matrix(df):
    sns.set_theme(style="white")
    corr = df.corr()
    mask = np.triu(np.ones_like(corr, dtype=bool))
    fig, ax = plt.subplots(figsize=(8, 8))
    # cmap = sns.diverging_palette(230, 20, as_cmap=True)
    sns.heatmap(corr, mask=mask, cmap="coolwarm", center=0,
                square=True, linewidths=.5, cbar_kws={"shrink": .5}, annot=True, fmt=".2f")

In [ ]:
plot_correlation_matrix(concrete_df)

Multicollinearity doesn't appear to be a problem with this dataset, so we'll retain all the existing features.

In [ ]:
def plot_pairplot(df):
    sns.set(font_scale=0.6)
    g = sns.pairplot(
        concrete_df, 
        diag_kind="kde", 
        height=0.9, 
        aspect=1.4, 
        plot_kws={'alpha': 0.2, 's': 5}
    )
    for i, j in zip(*plt.np.triu_indices_from(g.axes, 1)):
        g.axes[i, j].set_visible(False)

    for ax in g.axes.flat:
        ax.xaxis.label.set_rotation(0)
        ax.yaxis.label.set_rotation(90)



In [ ]:
plot_pairplot(concrete_df)

The "age" feature will need a log transform, which we'll do below.  Several other features exhibit a bimodal distribution, but with significant differences between major and minor modes.  We'll keep them as is for this example.

## Create Train and Test Sets

In [ ]:
train_set, test_set = train_test_split(concrete_df, test_size=0.2, random_state=42)

In [ ]:
label_col = "csMPa"
feature_cols = [col for col in concrete_df.columns if col != label_col]

In [ ]:
X_train, y_train = train_set[feature_cols], train_set[[label_col]]
X_test, y_test = test_set[feature_cols], test_set[[label_col]]

## Create Preprocessing Pipeline

In [ ]:
class LogTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None, sample_weight=None):
        self.feature_names_ = X.columns
        self.n_features_in_ = X.shape[1]
        return self

    def transform(self, X):
        check_is_fitted(self)
        X = check_array(X)
        assert self.n_features_in_ == X.shape[1]

        return np.log(X)

    def get_feature_names_out(self, names=None):
        return self.feature_names_
    
    def inverse_transform(self, X):
        return np.exp(X)


Created a custom class for the log transform to allow for the predict method to be used (this requires the inverse_transform method to be implemented)

In [ ]:
std_scaler = StandardScaler()
log_transformer = LogTransformer()

log_features = ["age"]
normal_features = [col for col in X_train.columns if col not in log_features]

preprocessing_pipeline = ColumnTransformer([
    ("standardize", std_scaler, normal_features),
    ("log_transform", log_transformer, log_features),
])

svm_reg_pipeline = Pipeline([
    ("preprocessing", preprocessing_pipeline),
    ("svm_reg", SVR()),
])

## Run a Randomized Search for Optimal Parameters Using SVM Regression Model

Parameter ranges are based on common parameter values for this model

In [ ]:
c_lower_bound = 0.01
c_upper_bound = 15.0
gamma_lower_bound = 0.001
gamma_upper_bound = 15.0

param_distributions = {
    "svm_reg__kernel": ["rbf", "linear"],
    "svm_reg__gamma": ["auto", "scale"],
    "svm_reg__C": uniform(loc=c_lower_bound, scale=c_upper_bound - c_lower_bound),
}

svr_rnd_search = RandomizedSearchCV(
    svm_reg_pipeline, 
    param_distributions=param_distributions, 
    n_iter=50, 
    scoring="neg_root_mean_squared_error", 
    cv=5, 
    verbose=0, 
    random_state=42
)

In [ ]:
svr_rnd_search.fit(X_train, y_train.values.ravel())

In [ ]:
svr_rnd_search.best_params_

In [ ]:
abs(svr_rnd_search.best_score_)

In [ ]:
svr_model = svr_rnd_search.best_estimator_

## Evaluate the Best Estimator on the Test Set

In [ ]:
y_pred = svr_model.predict(X_test)
rmse = mean_squared_error(y_test["csMPa"], y_pred, squared=False)
print(f"RMSE for best model = {rmse:.2f}")

In [ ]:
confidence = 0.95
squared_errors = (y_pred - y_test["csMPa"]) ** 2
confidence_interval = np.sqrt(
    stats.t.interval(
        confidence, 
        len(squared_errors) - 1, 
        loc=squared_errors.mean(), 
        scale=stats.sem(squared_errors)
    )
)

In [ ]:
print(f"RMSE Confidence interval = {confidence_interval[0]:.2} to {confidence_interval[1]:.2}")

Although the RMSE is slightly higher, the model doesn't appear to be significantly overfitting on the test set